In [1]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# Declare constants
training_source = 'Udemy'
tech_name = 'AWS'
no_of_courses = 6

# course id
aws_certified_cloud_practitioner = 0
aws_soultions_architect_associate = 1
aws_developer_associate = 2
aws_certified_sysops_administrator_associate = 3
aws_certified_devops_engineer_professional = 4
aws_certified_solutions_architect_professional=5

# Initial Arrays 
exam_names = ['']*no_of_courses
exam_codes = ['']*no_of_courses
urls =['']*no_of_courses

# Course names
exam_names[aws_certified_cloud_practitioner] ='aws-certified-cloud-practitioner'
exam_names[aws_developer_associate] ='aws-developer-associate'
exam_names[aws_soultions_architect_associate] ='aws-soultions-architect-associate' 
exam_names[aws_certified_sysops_administrator_associate]='aws-certified-sysops-administrator-associate'
exam_names[aws_certified_devops_engineer_professional]='aws-certified-devops-engineer-professional'
exam_names[aws_certified_solutions_architect_professional]='aws-certified-solutions-architect-professional'

# Exam Codes
exam_codes[aws_certified_cloud_practitioner] ='CLF-C01'
exam_codes[aws_developer_associate] ='DVA-C01'
exam_codes[aws_soultions_architect_associate] ='SAA-C02'  
exam_codes[aws_certified_sysops_administrator_associate]='SOA-C01'
exam_codes[aws_certified_devops_engineer_professional]='DOP-C01'
exam_codes[aws_certified_solutions_architect_professional]='SAP-C01'

# Course urls
urls[aws_certified_cloud_practitioner]=''

urls[aws_developer_associate] = 'https://www.udemy.com/course/aws-certified-developer-associate-dva-c01/'
urls[aws_soultions_architect_associate] ='https://www.udemy.com/course/aws-certified-solutions-architect-associate-saa-c02/'
urls[aws_certified_sysops_administrator_associate]=''

urls[aws_certified_devops_engineer_professional]=''
urls[aws_certified_solutions_architect_professional]=''

In [10]:
# Set Current Course & URL 
current_exam_scraped = aws_soultions_architect_associate
exam_code = exam_codes[current_exam_scraped]
exam_name = exam_names[current_exam_scraped]
start_url = urls[current_exam_scraped]

# Set the file name
file_name = training_source+'-'+exam_name

In [12]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(10)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)

In [13]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read() 

In [14]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('TECH')
table_columns.append('EXAM CODE')
table_columns.append('TRAINING PORTAL')
table_columns.append('EXAM NAME')
table_columns.append('COURSE NAME')
table_columns.append('TITLE ORDER')
table_columns.append('TITLE')
table_columns.append('NO OF SUB-SECTIONS')
table_columns.append('SUB TITLE')
table_columns.append('DURATION')

table_data = []
title_order = 0

# Grap the course name 
course_name = soup.select('h1[class="clp-lead__title clp-lead__title--small"]')[0].text.strip()

# Select course list
course_content = soup.select('div[class*="section--panel--"]')  
for section in course_content:
    # Set loop Variables
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    
    # Extract Data
    ele = section.select('div[class*="section--section--"]')[0]
    title = ele.select('div[class*="section--section-title--"]')[0].text
    section_content = ele.select('div[class*="udlite-text-sm section--section-content--"]')[0]
    section_content = section_content.get_text(separator=" ", strip=True).split('•',1)[0]
    section_duration = ele.select('span')[0].text
    #sub_sections = section.select('div[class*="section--lecture-title-and-description--"]')
    
    section_panels = section.select('div[class*="section--panel-content--"]')
    # Set row data
    row_data.append(tech_name)
    row_data.append(exam_code)
    row_data.append(training_source)
    row_data.append(exam_name)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(section_duration)
    table_data.append(row_data)
    
    sub_sections_count = 0
    for section_panel in section_panels:
        section_lectures = section_panel.select('div[class*="section--lecture-title-and-description--"]')
        for index,section_lecture in enumerate(section_lectures):
            sub_sections_count +=1
            sub_row = row_data.copy()
        
            sub_section = section_lecture
            sub_duration = section_panel.select('span[class*="section--lecture-content--"]')[index]
            sub_title = sub_section.select('span')[0]
        
            # set the rows values
            sub_row[8] = sub_title.text
            sub_row[9] = sub_duration.text
            table_data.append(sub_row)
    row_data[7] = sub_sections_count

In [15]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,TECH,EXAM CODE,TRAINING PORTAL,EXAM NAME,COURSE NAME,TITLE ORDER,TITLE,NO OF SUB-SECTIONS,SUB TITLE,DURATION
0,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,1,Introduction - AWS Certified Solutions Archite...,8,,15min
1,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,1,Introduction - AWS Certified Solutions Archite...,,Course Introduction - AWS Certified Solutions ...,03:01
2,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,1,Introduction - AWS Certified Solutions Archite...,,PLEASE READ: Lectures you can skip if you took...,00:57
3,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,1,Introduction - AWS Certified Solutions Archite...,,Creating an AWS Account,01:53
4,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,1,Introduction - AWS Certified Solutions Archite...,,AWS Account Activation Troubleshooting,02:05
...,...,...,...,...,...,...,...,...,...,...
337,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,26,Preparing for the Exam + Practice Exam - AWS C...,,Practice Exam 2 - Extra Questions,30 questions
338,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,27,Congratulations - AWS Certified Solutions Arch...,3,,3min
339,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,27,Congratulations - AWS Certified Solutions Arch...,,Congratulations - AWS Certified Solutions Arch...,00:48
340,AWS,SAA-C02,Udemy,aws-soultions-architect-associate,Ultimate AWS Certified Solutions Architect Ass...,27,Congratulations - AWS Certified Solutions Arch...,,THANK YOU!,01:32
